In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
!pip install -q pyyaml h5py

In [2]:
!pip install bert-tensorflow
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

     |████████████████████████████████| 71kB 2.2MB/s 



In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# this will give a link to your google drive, please click on the link and follow steps, which will give the verification code
# once verification code validated then , you should get the sharable link of the file and copy it in the below section

In [4]:
link = 'https://drive.google.com/open?id=1Bzn18bgU7dnr8nMhZvaRCQUtG_Dly3vt' 
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('drugsComTrain_raw.csv')  
Myfile_train = pd.read_csv('drugsComTrain_raw.csv')

link = 'https://drive.google.com/open?id=19sr2gCOKez5HQ4-ErnkDYK2JtZgqGUq8' 
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('drugsComTest_raw.csv')  
Myfile_test = pd.read_csv('drugsComTest_raw.csv')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
def Clean_Lables_inDS(Mydf):
  Mydf = Mydf[pd.notnull(Mydf['condition'])]
  Mydf.condition =Mydf.condition.str.replace(r"[^a-z'A-Z]+", " ")
  Mydf.condition=Mydf.condition.str.replace('<[^<]+?>', '')
  # replace those values in condition columns which has aggreagetd count less than 10
  s=Mydf['condition'].value_counts()
  Mydf['condition1'] = np.where(Mydf['condition'].isin(s.index[s >=10]), Mydf['condition'], 'Other')
  return(Mydf)


Myfile_train=Clean_Lables_inDS(Myfile_train)

Myfile_test=Clean_Lables_inDS(Myfile_test)



In [0]:
train_InputSet=Myfile_train.apply(lambda dsset: bert.run_classifier.InputExample(guid=None,text_a = dsset['review'], text_b = None, label = dsset['condition1']), axis = 1)

test_InputSet=Myfile_test.apply(lambda dsset: bert.run_classifier.InputExample(guid=None,text_a = dsset['review'], text_b = None, label = dsset['condition1']), axis = 1)



In [28]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [8]:

# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 100
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputSet, Myfile_train['condition1'], MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputSet, Myfile_train['condition1'], MAX_SEQ_LENGTH, tokenizer)
print(len(train_features))

INFO:tensorflow:Writing example 0 of 62780


INFO:tensorflow:Writing example 0 of 62780


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Writing example 10000 of 24874


INFO:tensorflow:Writing example 10000 of 24874


INFO:tensorflow:Writing example 20000 of 24874


INFO:tensorflow:Writing example 20000 of 24874


62780


In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
       
        return {
            "eval_accuracy": accuracy,
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
BATCH_SIZE =10
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 2.0
WARMUP_PROPORTION = 0.1
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
#num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_train_steps=100
#num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
num_warmup_steps=10


In [0]:
model_fn = model_fn_builder(
  num_labels=len(Myfile_train['condition1']),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  params={"batch_size": BATCH_SIZE})


In [0]:
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)


In [0]:

estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)


In [0]:

test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [17]:

estimator.evaluate(input_fn=test_input_fn, steps=None)




INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 100: eval_accuracy = 0.18131383, global_step = 100, loss = 8.584184


INFO:tensorflow:Saving dict for global step 100: eval_accuracy = 0.18131383, global_step = 100, loss = 8.584184


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: /tmp/tmprvhqe6vz/model.ckpt-100


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: /tmp/tmprvhqe6vz/model.ckpt-100


{'eval_accuracy': 0.18131383, 'global_step': 100, 'loss': 8.584184}

In [0]:
"### The below is sample file for prediction\n",
link = 'https://drive.google.com/open?id=1JpWHf1BdGinbLN_TkvYJC-mVkXvonK2B'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('drugsComTest_raw1.csv')
Mypredict_train = pd.read_csv('drugsComTest_raw1.csv')

In [0]:
Review_List= Mypredict_train['review'].tolist()
print(len(Review_List))
print(Myfile_train['condition1'][0])
predict_InputSet=[bert.run_classifier.InputExample(guid=None,text_a = x, text_b = None, label=Myfile_train['condition1'][0]) for x  in Review_List]
predict_features = run_classifier.convert_examples_to_features(predict_InputSet, Myfile_train['condition1'], MAX_SEQ_LENGTH, tokenizer)
predict_input_fn = run_classifier.input_fn_builder(features=predict_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
predictions = estimator.predict(predict_input_fn)
prediction_list=[(sentence, prediction['probabilities'], Myfile_train['condition1'][prediction['labels']]) for sentence, prediction in zip(Review_List, predictions)]

In [89]:
i=0
for x in  range(len(prediction_list)):
  if prediction_list[x][2]==Mypredict_train['condition'][x]:
    i=i+1
print("Accuracy:" + str(i/len(Review_List)*100))                                                                                                     


Accuracy:18.181818181818183
